In [380]:
# load up the test file
data = open("Test.neu").read()

# deal with newlines
data = data.split("\n")
for i, line in enumerate(data):
    data[i] = line.strip()
data = "".join(data)

In [381]:
cache = {
    "names": [],
    "cache": [],
    "objects": []
}

# parse a string entry according to its type and return a dict
def parse_entry(entry):
    if type(entry["value"]) == str:
        # convert values to proper types as needed
        if entry["type"]["name"] == "float":
            converted_value = float(entry["value"])
        elif entry["type"]["name"] == "int":
            converted_value = int(entry["value"])
        elif entry["type"]["name"] == "vector":
            converted_value = entry["value"].split(",")
            for i, value in enumerate(converted_value):
                converted_value[i] = float(value)
        else:
            converted_value = entry["value"]

        # return a dictionary of the parsed entry
        return {"type": entry["type"], "value": converted_value}
    else:
        return entry

# iterate through each character in the raw data and create a cache of parsed strings
stage = ""
entry = {"type": "", "value": ""}
consume = False
is_literal = False
is_reference = False
for char in data:
    if char == '"':
        # enter/exit literal consumption (handle special characters in values)
        is_literal = not is_literal
        continue
    # ignore special characters if they're within a literal value
    if not is_literal:
        if char == "@":
            stage = "names"
            continue
        elif char == "*":
            stage = "objects"
            continue
        elif char == "#":
            stage = "cache"
            continue
        elif char == "{":
            # start consuming at the start of each stage
            consume = True
            continue
        elif char == "}":
            # commit entry
            if stage != "names":
                # parse into proper values
                entry = parse_entry(entry)
                cache[stage].append(entry)
                entry = {"type": "", "value": ""}
            else:
                cache[stage].append(entry["value"])
                entry["value"] = ""

            is_reference = False

            # stop consuming at the end of each stage
            consume = False
        elif char == ",":
            # commit entry
            if stage != "names":
                # parse into proper values
                entry = parse_entry(entry)
                cache[stage].append(entry)
                entry = {"type": "", "value": ""}
            else:
                cache[stage].append(entry["value"])
                entry["value"] = ""
            
            is_reference = False

            continue
        elif char == ":":
            # handle names/types
            # handle nested types
            if "=" in entry["value"]:
                type_parent = cache["names"][int(entry["value"][0])]
                type_children = entry["value"][2:].split("/")
                for i, child in enumerate(type_children):
                    child_name = cache["names"][int(child.split(".")[0])]
                    child_type = cache["names"][int(child.split(".")[1])]
                    type_children[i] = {"name": child_name, "type": child_type}
                this_type = {
                    "name": type_parent,
                    "parameters": type_children
                }
                entry["type"] = this_type
            else:
                this_type = entry["value"]
                entry["type"] = {"name": cache["names"][int(this_type, base = 16)], "parameters": []}
            entry["value"] = ""
            continue
        elif char == "&":
            is_reference = True
            continue

    if consume:
        # handle cache references
        if is_reference:
            cached_object = cache["cache"][int(char, base = 16)]
            entry = cached_object
            continue
        else:
            entry["value"] += char

AttributeError: 'dict' object has no attribute 'split'

In [ ]:
graph = cache["objects"]

In [ ]:
# DEBUG
import json
with open("NeutrinoTest_Cache.json", "w") as outfile:
    json.dump(graph, outfile, indent = 4)